# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [73]:
from typing import List, Tuple, Dict, Callable

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [74]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> List[Tuple]:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return the empty List (`[]`).


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

### dominated_cols
Takes the current game state and returns a set of indices of columns that are dominated. A column is dominated if it is either weakly or strictly dominated by another column. **Used by**: [generate_next_states](#generate_next_states)

* **game** List[List[Tuple[int, int]]]:  The game matrix where each entry is a tuple of payoffs for each player.
* **weak** bool: Whether to use weak domination

**returns** A set of indices of dominated columns

In [75]:
def dominated_cols(game: List[List[Tuple]], weak: bool=False) -> set:
    cols = [] 
    for i in range(len(game[0])):
        col = [game[j][i][1] for j in range(len(game))]
        cols.append(col)
    
    dominated = set()
    for i in range(len(cols)):
        for j in range(len(cols)):
            if i != j:
                if weak:
                    if all(cols[i][k] <= cols[j][k] for k in range(len(cols[i]))) and any(cols[i][k] < cols[j][k] for k in range(len(cols[i]))):
                        dominated.add(i)
                else:
                    if all(cols[i][k] < cols[j][k] for k in range(len(cols[i]))):
                        dominated.add(i)
    return dominated

In [76]:
assert dominated_cols(prisoners_dilemma) == {1}
assert dominated_cols([[(1, 1), (3, 2)], [(5, 6), (7, 6)]]) == set()
assert dominated_cols([[(1, 1), (3, 2)], [(5, 6), (7, 6)]], True) == {0}

### dominated_rows
`dominated_rows` Takes the current game state and returns a set of indices of rows that are dominated. A row is dominated if it is either weakly or strictly dominated by another row. **Used by**: [generate_next_states](#generate_next_states)

* **game** List[List[Tuple[int, int]]]:  The game matrix where each entry is a tuple of payoffs for each player.
* **weak** bool: Whether to use weak domination

**returns** A set of indices of dominated columns

In [77]:
def dominated_rows(game: List[List[Tuple]], weak: bool=False) -> set:
    rows = []
    for i in range(len(game)):
        row = game[i]
        rows.append(row)

    dominated = set()
    for i in range(len(rows)):
        for j in range(len(rows)):
            if i != j:
                if weak:
                    if all(rows[i][k][0] <= rows[j][k][0] for k in range(len(rows[i]))) and any(rows[i][k][0] < rows[j][k][0] for k in range(len(rows[i]))):
                        dominated.add(i)
                else:
                    if all(rows[i][k][0] < rows[j][k][0] for k in range(len(rows[i]))):
                        dominated.add(i)
    return dominated

In [78]:
assert dominated_rows(prisoners_dilemma) == {1}
assert dominated_rows([[(1, 2), (3, 4)], [(5, 6), (3, 7)]]) == set()
assert dominated_rows([[(1, 2), (3, 4)], [(5, 6), (3, 7)]], True) == {0}

### remove_col
`remove_col` Removes a column from the game matrix. **Used by**: [generate_next_states](#generate_next_states)

* **game** List[List[Tuple[int, int]]]:  The game matrix where each entry is a tuple of payoffs for each player.
* **col** int: The index of the column to remove.

**return** A new game matrix with the specified column removed.

In [79]:
def remove_col(game: List[List[Tuple]], col: int) -> List[List[Tuple]]:
    new_game = []
    for row in game:
        new_row = [cell for i, cell in enumerate(row) if i != col]
        new_game.append(new_row)
    return new_game

In [80]:
assert remove_col([[(1, 2), (3, 4)], [(5, 6), (3, 7)]], 0) == [[(3, 4)], [(3, 7)]]
assert remove_col([[(1, 2), (3, 4)], [(5, 6), (3, 7)]], 1) == [[(1, 2)], [(5, 6)]]
assert remove_col([[(1, 2), (3, 4)], [(5, 6), (3, 7)]], 2) == [[(1, 2), (3, 4)], [(5, 6), (3, 7)]]

### remove_row
`remove_row` Remove row from the game matrix. **Used by**: [generate_next_states](#generate_next_states)

* **game** List[List[Tuple[int, int]]]:  The game matrix where each entry is a tuple of payoffs for each player.
* **col** int: The index of the row to remove.

**return** A new game matrix with the specified row removed.

In [81]:
def remove_row(game: List[List[Tuple]], row: int) -> List[List[Tuple]]:
    return [game[i] for i in range(len(game)) if i != row]

In [82]:
assert remove_row([[(1, 2), (3, 4)], [(5, 6), (3, 7)]], 0) == [[(5, 6), (3, 7)]]
assert remove_row([[(1, 2), (3, 4)], [(5, 6), (3, 7)]], 1) == [[(1, 2), (3, 4)]]
assert remove_row([[(1, 2), (3, 4)], [(5, 6), (3, 7)]], 2) == [[(1, 2), (3, 4)], [(5, 6), (3, 7)]]

### generate_next_states
`generate_next_states` takes the current game state and generates the next possible states based on the previous move. **Used by**: [solve_game](#solve_game)

* **game** List[List[Tuple[int, int]]]:  The game matrix where each entry is a tuple of payoffs for each player.
* **prev_move** str: The last move made, either "row", "col" or None.
* **row_map** List[int]: A mapping of the original rows to the current state.
* **col_map** List[int]: A mapping of the original columns to the current state.
* **weak** bool: Whether to use weak domination

**returns** A list of tuples where each tuple contains a new game state, the type of move made ("row" or "col"), and the updated row and column mappings.

In [83]:
def generate_next_states(game: List[List[Tuple]], prev_move: str, row_map: List[int], col_map: List[int], weak: bool=False) -> List[Tuple[List[List[Tuple]], str, List[int], List[int]]]:
    next_states = []
    if prev_move == "row":
        dominated = dominated_cols(game, weak)
        if dominated:
            for col in dominated:
                next_state = remove_col(game, col)
                next_states.append((next_state, "col", row_map, col_map[:col] + col_map[col+1:]))
    
    elif prev_move == "col":
        dominated = dominated_rows(game, weak)
        if dominated:
            for row in dominated:
                next_state = remove_row(game, row)
                next_states.append((next_state, "row", row_map[:row] + row_map[row+1:], col_map))
    else:
        rows = dominated_rows(game, weak)
        cols = dominated_cols(game, weak)
        for row in rows:
            next_state = remove_row(game, row)
            next_states.append((next_state, "row", row_map[:row] + row_map[row+1:], col_map))
        for col in cols:
            next_state = remove_col(game, col)
            next_states.append((next_state, "col", row_map, col_map[:col] + col_map[col+1:]))

    return next_states

### solve_game
`solve_game` Solves the game by applying Successive Elimination of Dominated Strategies (SEDS) to find the pure strategy Nash Equilibrium of a Normal Form Game.

* **game** List[List[Tuple[int, int]]]:  The game matrix where each entry is a tuple of payoffs for each player.
* **weak** bool: Whether to use weak domination

**returns** A list of tuples representing the Nash Equilibria of the game.

In [84]:
def solve_game(game: List[List[Tuple]], weak: bool=False) -> List[Tuple]:
    row_map = [i for i in range(len(game))]
    col_map = [i for i in range(len(game[0]))]
    next_states = generate_next_states(game, None, row_map, col_map, weak)
    equilibriums = set()

    while next_states:
        current_state, prev_move, row_map, col_map = next_states.pop(0)

        next_states += generate_next_states(current_state, prev_move, row_map, col_map, weak)

        if len(current_state) == 1 and len(current_state[0]) == 1:
            equilibriums.add((col_map[0], row_map[0]))

    return list(equilibriums)
    

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices (a list of them).

## Before you code...

Solve the following game by hand using SEDS and weakly dominated strategies. 
The game has two (pure) Nash Equilibriums. 
You should find all of them.
This will help you think about what you need to implement to make the algorithm work.
**Hint**: You will need State Space Search from Module 1 and SEDS from Module 5 to get the full algorithm to work.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions**: [(1, 2), (2, 0)]

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solution:** {(2, 0), (1, 2)}

In [85]:
test_game_1 = [
    [(2, 1), (1, 0), (0, 2)],
    [(3, 2), (2, 1), (1, 0)],
    [(1, 3), (0, 1), (2, 1)],
]

solution = solve_game(test_game_1, weak=False)

In [86]:
assert solution == [(0, 1)] # insert your solution from above. TODO: change to return indices instead of values

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 2/1 | 3/0 | 4/2 |
|1  | 1/1 | 3/1 | 3/1 |
|2  | 0/1 | 1/1 | 2/0 |

**Solution:** [(2, 0)]

In [87]:
test_game_2 = [
    [(2, 1), (3, 0), (4, 2)],
    [(1, 1), (3, 1), (3, 1)],
    [(0, 1), (1, 1), (2, 0)],
]

strong_solution = solve_game( test_game_2)
weak_solution = solve_game( test_game_2, weak=True)

In [88]:
assert strong_solution == []
assert weak_solution == [(2, 0)]

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/1 | 0/2 | 3/0 |
|1  | 0/0 | 2/1 | 1/3 |
|2  | 2/3 | 1/0 | 0/2 |

**Solution:** None

In [89]:
test_game_3 = [
    [(1, 1), (0, 2), (3, 0)],
    [(0, 0), (2, 1), (1, 3)],
    [(2, 3), (1, 0), (0, 2)],
]

strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

In [90]:
assert strong_solution == []
assert weak_solution == []

### Test Game 4. Multiple Equilibria

You solved the following game by hand, above.
Now use your code to solve it.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions:** [(1, 2), (2, 0)]

In [91]:
test_game_4 = [
[(1, 0), (3, 1), (1, 1)],
[(1, 1), (3, 0), (0, 1)],
[(2, 2), (3, 3), (0, 2)]]

strong_solution = solve_game( test_game_4)
weak_solution = solve_game( test_game_4, weak=True)

In [92]:
assert strong_solution == []
assert weak_solution == [(1, 2), (2, 0)]

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.